In [12]:
import pandas as pd
import re


# Lectura del Dataset

Iniciamos con la lectura del dataset, aquí es muy importante desde el inicio tener claro, que los datos que nos interesan son aquellas en las que tengan el "inbound=True", porque son aquellos mensajes que el usuario hace hacia la empresa. El objetivo principal para lo que se planea usar los siguientes datos, es clasificar los mensajes de los usuarios dependiendo si es critico o no, para facilitar el acceso de los twets al soporte de la empresa.

In [13]:
df = pd.read_csv('twcs.csv')

df = df[df['inbound'] == True] # Importante! Esto solo nos dejara los tweets de los usuarios a la empresa

df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


Como podemos observar en los primeros elementos del dataset, la única columna que en el momento nos interesa es "text", porque este contiene el mensaje del usuario. Como en este caso no tenemos una columna "Label", o la columna que predecir, mas adelante veremos como preparamos el dataset, para tener dos etiquetas: 0->No crítico, es decir los mensajes en los que posiblemente no haya que responder al usuario. 1->Crítico, mensaje que es necesario responder al usuario, ya que posiblemente puede ser alguna queja, problema, error, que toque darle soporte al usuario.

## Analisis Exploratorio

In [14]:
print("Numero de tweets: ", len(df))

Numero de tweets:  1537843


In [15]:
df.isnull().sum()                   # Nulos por columna

tweet_id                        0
author_id                       0
inbound                         0
created_at                      0
text                            0
response_tweet_id          234014
in_response_to_tweet_id    787346
dtype: int64

In [16]:
df['text'].str.len().describe()     # Estadísticas de longitud de texto

count    1.537843e+06
mean     1.099361e+02
std      5.716449e+01
min      1.000000e+00
25%      6.700000e+01
50%      1.090000e+02
75%      1.400000e+02
max      5.130000e+02
Name: text, dtype: float64

De este pequeño analisis exploratorio, podemos observar, que no hay que lidiar con datos nulos, porque como se mencionó anteriormente, de aquí la única columna que nos interesa es la de "text"

## Limpieza de datos

En esta sección, la idea principal es estandarizar y limpiar un poco el texto, para luego poder etiquetar.

In [17]:
def clean_text(text):
    text = text.lower() # Convertir a minúsculas
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Eliminar URLs
    return text.strip()

df['text'] = df['text'].apply(clean_text) # Aplicar la limpieza a cada tweet

In [18]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare i have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare i did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare you gonna magically change your co...,"11,13,14",15.0


## Etiquetado de tweets

En esta sección, se hace un etiquetado rápido de los datos, se usan algunas palabras claves que usualmente en este tipo de problemas se necesitan identificar. Ya si se quiere hacer un etiquetado robusto, se recomienda usar la tecnica de Topic Modeling, que permite identificar nuevas categorias y mejorar la clasificación.

In [19]:
def label_message(text):
    critical_words = ['problem', 'issue', 'not working', 'refund', 'error', 'fail', 'help', 'wtf', 'worst', 'urgent', 'bad']
    for word in critical_words:
        if word in text.lower():
            return 'critical'
    return 'non_critical'

In [20]:
df['label'] = df['text'].apply(label_message)
df['label'].value_counts()         # Contar la cantidad de tweets por etiqueta

label
non_critical    1052276
critical         485567
Name: count, dtype: int64

# Conclusión

En este punto, se termina la exploración inicial de los datos y un prototipo de etiquetado, una de las cosas mas importantes que se hace en este notebook, es descubrir que en la columna que nos interesa `text` no tiene valores nulos, y que podemos etiquetar de una manera rápida, aquellos mensajes que posiblemente tienen palabras claves que usualmente son de urgencia o que necesitan apoyo de soporte.